In [1]:
pip install langchain PyPDF2 openai faiss-cpu


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install langchain-community

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# 1. PDF Data Ingestion
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def split_text_into_chunks(text, chunk_size=500, overlap=50):
    """Splits text into smaller chunks for vectorization."""
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i : i + chunk_size])
    # print(chunks)
    return chunks

# 2. Vector Database and Embedding
def create_vector_store(chunks, model_name):
    """Converts text chunks into vector embeddings and stores them in FAISS."""
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks)

    # Initialize FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index, embeddings, chunks

def query_vector_store(index, embeddings, chunks, query, model_name, top_k=3):
    """Queries the vector store to retrieve relevant chunks."""
    model = SentenceTransformer(model_name)
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, top_k)
    return [chunks[i] for i in indices[0]]

# 3. LLM for Response Generation
def generate_response(retrieved_chunks, query):
    """Generates a response using an open-source LLM."""
    context = "\n".join(retrieved_chunks)
    prompt = f"Answer the question based on the following context:\n{context}\n\nQuestion: {query}"

    # Using Hugging Face Transformers pipeline for LLM response
    qa_pipeline = pipeline("text-generation", model="gpt2")  # Replace with a larger model like GPT4All
    # Use max_new_tokens instead of max_length to specify the desired length for the generated response
    response = qa_pipeline(prompt, max_new_tokens=200, num_return_sequences=1)
    return response[0]["generated_text"]
# Example Workflow

pdf_path = "C:\Users\manil\Desktop\sithafal\ORDER DATA.pdf"  # Replace with your PDF file path
model_name = "all-MiniLM-L6-v2"  # Open-source embedding model

# Step 1: Extract text from PDF and split into chunks
text = extract_text_from_pdf(pdf_path)
chunks = split_text_into_chunks(text)

# Step 2: Create vector store
vector_store, embeddings, chunks = create_vector_store(chunks, model_name)



SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (172873762.py, line 56)

In [5]:
# Step 3: Handle user query
query = "who is team lead of this project?"
retrieved_chunks = query_vector_store(vector_store, embeddings, chunks, query, model_name)

# Step 4: Generate response
response = generate_response(retrieved_chunks, query)
print("Response:", response)



NameError: name 'query_vector_store' is not defined

In [6]:
import os
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1. PDF Data Ingestion
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

def split_text_into_chunks(text, chunk_size=500, overlap=50):
    """Splits text into smaller chunks for vectorization."""
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i : i + chunk_size])
    return chunks

# 2. Vector Database and Embedding
def create_vector_store(chunks, model_name):
    """Converts text chunks into vector embeddings and stores them in FAISS."""
    model = SentenceTransformer(model_name)
    embeddings = model.encode(chunks)

    # Initialize FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index, embeddings, chunks

def query_vector_store(index, chunks, query, model_name, top_k=5):
    """Queries the vector store to retrieve relevant chunks."""
    model = SentenceTransformer(model_name)
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, top_k)
    return [chunks[i] for i in indices[0]]

# 3. LLM for Response Generation
def generate_response(retrieved_chunks, query, model_name="google/flan-t5-large"):
    """Generates a response using an open-source LLM."""
    context = "\n".join(retrieved_chunks)
    prompt = f"Answer the question based on the following context:\n\n{context}\n\nQuestion: {query}"

    # Load Hugging Face model for text generation
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    outputs = model.generate(
        inputs.input_ids, max_length=300, num_return_sequences=1, temperature=0.3
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example Workflow

# Replace with your PDF file path
pdf_path = "/content/MINI PROJECT FINAL DOCUMENTATION __2024.pdf"  # Ensure this is a valid PDF file in the working directory
embedding_model = "all-MiniLM-L6-v2"  # Open-source embedding model

# Step 1: Extract text from PDF and split into chunks
print("Extracting text from PDF...")
text = extract_text_from_pdf(pdf_path)
chunks = split_text_into_chunks(text)

# Step 2: Create vector store
print("Creating vector store...")
vector_store, embeddings, chunks = create_vector_store(chunks, embedding_model)
print(vector_store)


ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:

# Step 3: Handle user query
query = "Machine Learning Libraries"  # Example query
print("Retrieving relevant chunks...")
retrieved_chunks = query_vector_store(vector_store, chunks, query, embedding_model)

# Step 4: Generate response
print("Generating response...")
response = generate_response(retrieved_chunks, query)
print("Response:", response)

